## Salary prediction for Kaggle Data Scientists

In this exercise, we will explore the data from the 2017 Kaggle survey. The goal is to find what is the most important criterion for the salary of a data scientist. The data is available [here](https://kermorvant.github.io/ml/data/multipleChoiceResponses.csv)

**Questions:**
> * Load the Kaggle dataset with `pd.read_csv('multipleChoiceResponses.csv')` 
> * print a few lines of the dataset (`df.head()`) and compute basic statistics (`df.describe()`)


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('multipleChoiceResponses.csv',encoding="ISO-8859-1",low_memory=False) # low_memory=False to remove warning
# you code here to print a few lines of the dataset  and compute basic statistics (df.describe())

Since we want to predict the salary (CompensationAmount), we keep only the responses which include a value for the salary and also the currency (CompensationCurrency) used for the salary. 

**Questions:**
> * use the function `df.dropna()` to only keep the examples in the dataframe for which `CompensationAmount` and `CompensationCurrency` are not null.


In [ ]:
# We need salary and currency
df.dropna(subset=['None','None'],inplace=True) # Your code here
print (df.shape)
print (df[['CompensationAmount','CompensationCurrency']][:10])

The salary values are not normalized : some include the thousand separator ',' and some include a space. We need to normalize the values : 
* remove the ","
* convert to numeric with `pd.to_numeric``


In [ ]:
# pd.to_numeric with coerce error to Not a number (Nan)
df['Salary']=pd.to_numeric(df['None'].str.replace(',',''),errors='coerce')  # Your code here, replace None
df['Salary'].isnull().sum()
print ("Could not convert ",df['Salary'].isnull().sum(),"values")

### Currency conversion

The salary is not always given in USD, so we need to convert all the amounts in USD. A conversion table is given [here](https://kermorvant.github.io/ml/data/conversionRates.csv). We will 

**Question** 

>* load this file and create a function to convert a tuple (amout,currency) in USD
>* apply this function to the dataframe with the function `apply(func,axis=1)`


In [ ]:
# load the conversion values
df_converter = pd.read_csv('conversionRates.csv',index_col=0)
df_converter.head()

In [ ]:
# define currency conversion function
def convert(x):
    """
    x[0] is the amount
    x[1] is the currency
    """
    return x[0]*df_converter.loc[x[1]]['exchangeRate']
print ("1 USD =",convert ([1,'USD']),'USD')
print ("1 euro=",convert ([1,'EUR']),'USD')

In [ ]:
# Then we apply the conversion function to the dataframe to create a new column with the salary in USD
df['SalaryUSD'] = df[['None','None']].apply(convert,axis=1)# Your code here, replace None
df['SalaryUSD'].describe()

### Outlier removal

The salary data is not ready yet ! We see that we have negative salaries and salaries greater than 1M USD. These values are outliers, they must be removed. We will use the percentile to remove the outliers.

**Question** 

>* compute the minial and maximal values to remove outliers using quantile at 10% and 90% (0.1 and 0.9)
>* filter the dataframe to keep only the salarie between these values

In [ ]:
maxval = df['None'].quantile(None)# Your code here, replace None
minval = df['None'].quantile(None)# Your code here, replace None
print ("Salary will be between ",int(minval),'USD and ',int(maxval),'USD')

df = df[(df['None'] < maxval) &(df['None'] > minval)]# Your code here, replace None
print ('Dataframe size after outlier removal:',df.shape)
df.SalaryUSD.describe()
# plot the historgram of frequences
_ = df.SalaryUSD.plot.hist()


### Variables exploration

The dataframe has a lot of variables : 230. We will focus on a subset of these variables : ['Age','GenderSelect','Country','EmploymentStatus','CurrentJobTitleSelect','MajorSelect']

Since the categorical data are not described with the function `df.describe()`, we will plot the data distribution for each of these variable.


**Question**

>* Plot the distribution for `GenderSelect`, `CurrentJobTitleSelect` and `MajorSelect`   values with `sns.countplot(data=df,y='GenderSelect')`
>* Plot the distribution for `EmploymentStatus` with `df.EmploymentStatus.value_counts().plot.pie()`


In [ ]:
_ = sns.countplot(data=df,y='GenderSelect')


In [ ]:
plt.figure(figsize=(15,10))
_ = sns.countplot(data=df,y='Country',order = df['Country'].value_counts().index)

In [ ]:
_ = df.EmploymentStatus.value_counts().plot.pie(figsize=(8, 8))

In [ ]:
_ = sns.countplot(data=df,y='CurrentJobTitleSelect',order = df['CurrentJobTitleSelect'].value_counts().index)

In [ ]:
_  = sns.countplot(data=df,y='MajorSelect',order = df['MajorSelect'].value_counts().index)

### Missing data imputation

Some of the values for the selected variables may be missing. We could ignore all the example with missing variable but it would reduce the size of the dataset. The best way to keep the sample with missing values is to fill the missing values. Many technics are possible, but in our case, we wil simply fill the missing values of the age with the mean age.

**Question**

>* Compute the mean age with `df.Age.mean()` and fill the missing values of Age with this value using `df['Age'].fillna(value)`
>* remove outliers using percentile (as before) at 1% and 99%

In [ ]:
mean_age = None # Your code here
print ("before filling, number of missing values is",df['Age'].isnull().sum())
df['Age'].fillna(None,inplace=True)
print ("after filling, number of missing values is",df['Age'].isnull().sum())
df['Age'].describe()


In [ ]:
maxval = df['Age'].quantile(None) # Your code here
minval = df['Age'].quantile(None) # Your code here
print ("Age will be between ",int(minval),' and ',int(maxval))

df = df[(df['Age'] < maxval) &(df['Age'] > minval)]
df.shape
_ = df.Age.plot.hist()

### Categorical data encoding

The categorical data can not be directly used in most classifiers, their values must be encoded for example with an integer associated to each different category. 
> * Encode the categorical data with a [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Missing values must be filled before fitting the encoder (use `fillna('UNK')` on each column).
> * print a few lines of the dataset to see the effect of the encoder.


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for c in ['GenderSelect','Country','EmploymentStatus','CurrentJobTitleSelect','MajorSelect']:
    df[c+'_label'] = le.fit_transform(df[c].fillna('UNK'))
    # print the encoded values
    print (c)
    print ([(idx,val) for idx,val in  enumerate(le.classes_)])
    print()
df.head()

### Regression tree training and visualisation

A decision tree can be used both for classification of for regression. In our case, since we want to predict the salary of a data scientist, we will use decision trees for regression.
> * Define the features X : 'Age', 'GenderSelect_label', 'Country_label', 'EmploymentStatus_label', 'CurrentJobTitleSelect_label', 'MajorSelect_label' and the target y 'SalaryUSD'
> * Fit a decision [DecisionTreeRegressor](http://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html) to predict the Salary. Set max_depth to 3 to visualize the tree. You can try other values.


In [ ]:
X = df[[None]] # Your code here
y = df['None'] # Your code here

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(max_depth=None) #Your code here
regr.fit(None, None) # Your code here


The tree can be display to see which rules have been used. The follwing functions plot the decision tree previously trained. 

In [ ]:
import graphviz
from sklearn import tree
dot_data = tree.export_graphviz(regr, out_file=None,feature_names=['Age','GenderSelect_label','Country_label','EmploymentStatus_label','CurrentJobTitleSelect_label','MajorSelect_label'])
graph = graphviz.Source(dot_data)  
graph 

**Question**:
>  what is the first splitting node ?

### Salary prediction

We will now build a model to predict the salary form the selected features. 

**Questions:**
> * split the dataframe in train and test set
> * train a RandomForestRegressor and a ExtraTreesRegressor
> * compare the 2 models with a metric adapted to regression : mean_absolute_error and explained_variance_score

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score


# split the dataframe in train and test
X_train, X_test, y_train, y_test = train_test_split(None, None, test_size=0.2, random_state=32)# YOUR CODE HERE

# train a random forest regressor
forest = RandomForestRegressor()
forest.fit(None, None)# YOUR CODE HERE

# evaluate the random forest with regression metrics
y_pred = forest.predict(None)# YOUR CODE HERE
print ('Mean absolute error:',mean_absolute_error(None, None))# YOUR CODE HERE
print ('Explained variance score:',explained_variance_score(None, None))# YOUR CODE HERE


etree = ExtraTreesRegressor()
etree.fit(X_train, y_train)
ey_pred = etree.predict(X_test)
print ('Mean absolute error:',mean_absolute_error(None, None))# YOUR CODE HERE
print ('Explained variance score:',explained_variance_score(None, None))# YOUR CODE HERE

### Feature importance

A RandomForest model can also be used to find the most important features in classification or regression problems. The importance of a feature is related to the number of times it is used in the different trees of the random forest.

The following code (adapted form [this code](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)) plot the feature importance on the Kaggle problem. 

**Question**
> * What is the most important feature to predict the salary ?
> 

In [ ]:

feature_names=['Age','GenderSelect_label','Country_label','EmploymentStatus_label','CurrentJobTitleSelect_label','MajorSelect_label']
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d %s(%f)" % (f + 1, indices[f],feature_names[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

### Predict you salary

You can use the forest to predict you salary (according to Kaggle standards): 

**Question**    
> * create a list with your parameters `['Age','GenderSelect_label','Country_label','EmploymentStatus_label','CurrentJobTitleSelect_label','MajorSelect_label']` encoded with the values previously defined with the LabelEncoder()
> * predict with the forest 

In [ ]:
#['Age','GenderSelect_label','Country_label','EmploymentStatus_label','CurrentJobTitleSelect_label','MajorSelect_label']]

# use np.array and reshape to avoid warning 
x = np.array([None]).reshape(1,-1)  # Your code here
forest.predict(x)